In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog

# 读取数据
land_info = pd.read_excel("D:/pythonproject/data_science/excel_files/附件1.xlsx", sheet_name="乡村的现有耕地")
crop_info = pd.read_excel("D:/pythonproject/data_science/excel_files/附件1.xlsx", sheet_name="乡村种植的农作物")
planting_2023 = pd.read_excel("D:/pythonproject/data_science/excel_files/附件2.xlsx", sheet_name="2023年的农作物种植情况")
stats_2023 = pd.read_excel("D:/pythonproject/data_science/excel_files/附件2.xlsx", sheet_name="2023年统计的相关数据")

# 预处理数据
# 提取地块类型和面积
land_types = land_info[['地块名称', '地块类型', '地块面积/亩']].dropna()

# 提取作物信息
crop_types = crop_info[['作物编号', '作物名称', '作物类型', '种植耕地']].dropna()

# 提取2023年种植情况
planting_2023 = planting_2023[['地块名称', '作物编号', '作物名称', '作物类型', '种植面积/亩', '种植季次']].dropna()

# 提取2023年统计数据
stats_2023 = stats_2023[['作物编号', '作物名称', '地块类型', '种植季次', '亩产量/斤', '种植成本/(元/亩)', '销售单价/(元/斤)']].dropna()

# 定义地块类型和作物类型的映射关系
land_type_mapping = {
    '平旱地': ['粮食'],
    '梯田': ['粮食'],
    '山坡地': ['粮食'],
    '水浇地': ['水稻', '蔬菜'],
    '普通大棚': ['蔬菜', '食用菌'],
    '智慧大棚': ['蔬菜']
}

# 定义作物轮作规则
# 每种作物不能连续重茬种植
# 每个地块三年内至少种植一次豆类作物

# 定义经济效益计算函数
def calculate_profit(crop_id, land_type, season, area):
    # 获取亩产量、成本和单价
    stats = stats_2023[(stats_2023['作物编号'] == crop_id) & 
                      (stats_2023['地块类型'] == land_type) & 
                      (stats_2023['种植季次'] == season)]
    if stats.empty:
        return 0
    yield_per_mu = stats['亩产量/斤'].values[0]
    cost_per_mu = stats['种植成本/(元/亩)'].values[0]
    price_range = stats['销售单价/(元/斤)'].values[0]
    # 处理价格范围
    if '-' in price_range:
        min_price, max_price = map(float, price_range.split('-'))
        price = (min_price + max_price) / 2
    else:
        price = float(price_range)
    # 计算利润
    revenue = yield_per_mu * price * area
    cost = cost_per_mu * area
    profit = revenue - cost
    return profit

# 定义线性规划模型
# 目标函数：最大化总利润
# 约束条件：地块类型限制、作物轮作要求、豆类作物种植频率等

# 定义变量：每种作物在每个地块每个季次的种植面积
# 由于变量较多，这里简化模型，仅展示框架
# 实际求解时需根据具体数据调整

# 定义目标函数系数
# 假设变量为种植面积，系数为每亩利润
# 这里仅示例，实际需根据数据计算
c = [-1, -2, -3]  # 示例系数

# 定义约束条件
# 地块面积约束
A = [[1, 1, 1]]
b = [100]  # 示例地块面积

# 求解线性规划问题
res = linprog(c, A_ub=A, b_ub=b, bounds=(0, None))

# 输出结果
print(res)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -300.0
              x: [ 0.000e+00  0.000e+00  1.000e+02]
            nit: 0
          lower:  residual: [ 0.000e+00  0.000e+00  1.000e+02]
                 marginals: [ 2.000e+00  1.000e+00  0.000e+00]
          upper:  residual: [       inf        inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00]
                 marginals: [-3.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0
